In [1]:
import pandas as pd
import os

full_path = os.path.join(os.getcwd(), 'theories')
dataframe = pd.read_excel(os.path.join(full_path, '1.xlsx'), usecols=['Preferred_Name', 'SMILES', 'InChIKey'])



In [2]:
area_dict = dict(zip(dataframe['Preferred_Name'], dataframe['SMILES']))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyError: 1

In [7]:
common_names = area_dict.keys()
smiles = area_dict.values()

In [8]:


######################### 2. PRE-PROCESS INPUT TEXT ###########################
from keras.preprocessing.text import Tokenizer

def tokenize(sentences, encode_start_end = False):

    if encode_start_end:
        sentences = ["startofsentence " + s + "endofsentence" for s in sentences]

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    tokenized_sentences = tokenizer.texts_to_sequences(sentences)

    return tokenized_sentences, tokenizer



######################### 2. PRE-PROCESS INPUT TEXT ###########################
from keras.preprocessing.text import Tokenizer

def tokenize(sentences, encode_start_end = False):

    if encode_start_end:
        sentences = ["startofsentence " + s + "endofsentence" for s in sentences]

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    tokenized_sentences = tokenizer.texts_to_sequences(sentences)

    return tokenized_sentences, tokenizer



Using TensorFlow backend.


In [9]:

from keras.preprocessing.sequence import pad_sequences

def pad(sentences, length = None):

    if length is None:
        length = max([len(s) for s in sentences])

    padded_sentences = pad_sequences(sentences,
                                     maxlen = length,
                                     padding = 'post',
                                     truncating = 'post')

    return padded_sentences


common_names_tokenized, common_names_tokenizer = tokenize(common_names)
smiles_tokenized, smiles_tokenizer = tokenize(smiles,
                                        encode_start_end = True)

common_names_encoded = pad(common_names_tokenized)
smiles_encoded = pad(smiles_tokenized)

common_names_size = len(common_names_tokenizer.word_index)
smiles_size = len(smiles_tokenizer.word_index)

common_names_seq_len = len(common_names_encoded[0])
smiles_seq_len = len(smiles_encoded[0])



English vocabulary size:  59703
frenish vocabulary size:  13526

Length of longest English sentence:  58
Length of longest frenish sentence:  440
3


In [10]:


######################### 3. BUILD *TRAINING* MODEL ###########################
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

encoder_input = Input(shape = (None, ),
                      name = "Encoder_Input")

embedding_dim = 200
embedded_input = Embedding(input_dim = common_names_size,
                           output_dim = embedding_dim,
                           name = "Embedding_Layer")(encoder_input)

encoder_lstm = LSTM(units = 256,
                    activation = "relu",
                    return_sequences = False,
                    return_state = True,
                    name = "Encoder_LSTM")

_, last_h_encoder, last_c_encoder = encoder_lstm(embedded_input)

decoder_input = Input(shape = (None, 1),
                      name = "Deocder_Input")

decoder_lstm = LSTM(units = 256,
                    activation = "relu",
                    return_sequences = True,
                    return_state = True,
                    name = "Decoder_LSTM")

all_h_decoder, _, _ = decoder_lstm(decoder_input,
                                   initial_state = [last_h_encoder, last_c_encoder])

final_dense = Dense(smiles_size,
                    activation = 'softmax',
                    name = "Final_Dense_Layer")

logits = final_dense(all_h_decoder)

seq2seq_model = Model(input = [encoder_input, decoder_input],
                      output = logits)

seq2seq_model.compile(loss = sparse_categorical_crossentropy,
                      optimizer = Adam(lr = 0.002),
                      metrics = ['accuracy'])


/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("Fi...)`


In [13]:
############################ 4. TRAIN THE MODEL ###############################
# Decoder: input - all but last word, target - all but "starofsentence" token
decoder_smiles_input = common_names_encoded.reshape((-1, common_names_seq_len, 1))[:, :-1, :]
decoder_smiles_target = smiles_encoded.reshape((-1, smiles_seq_len, 1))[:, 1:, :]

seq2seq_model.fit([common_names_encoded, decoder_smiles_input],
                  decoder_smiles_target,
                  epochs = 16,
                  batch_size = 1024)


/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/16
